In [ ]:
import os
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

from data_dengue import baixar_dados_por_estado
from feature_dengue import carregar_e_limpar, adicionar_variaveis_temporais, remover_outliers
from modelagem import treinar_modelos

if __name__ == "__main__":
    # Baixa os dados por estado
    baixar_dados_por_estado(diretorio_saida="data/raw", ano_inicio=2022, ano_fim=2025)

    # Carrega e processa os dados
    df = carregar_e_limpar("data/raw/dengue_2022_2025_por_estado.csv")
    print("Colunas carregadas:")
    print(df.columns.tolist())

    df_feat = adicionar_variaveis_temporais(df)
    features = ["casos_lag_1", "casos_media_3", "pop", "tempmin", "umidmax", "receptivo", "transmissao"]
    df_feat_sem_outliers = remover_outliers(df_feat, features)

    # Cria diretórios de saída
    output_dir = "data/processed"
    figures_dir = "figures"
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(figures_dir, exist_ok=True)

    # Salvar e recarregar os dados processados
    df_feat_sem_outliers.to_csv(os.path.join(output_dir, "dengue_2022_2025_features_sem_outliers.csv"), index=False, sep=";")
    df_feat_sem_outliers = pd.read_csv(os.path.join(output_dir, "dengue_2022_2025_features_sem_outliers.csv"), sep=";")

    X = df_feat_sem_outliers[["casos_lag_1", "casos_media_3", "pop", "tempmin", "umidmax", "receptivo", "transmissao"]]
    y = df_feat_sem_outliers["casos"]

    # Treinamento dos modelos
    resultados = treinar_modelos(X, y)

    # Avaliação dos modelos
    for modelo_nome, (mse, r2) in resultados.items():
        mse_percentual = (mse / y.mean()) * 100
        r2_percentual = r2 * 100
        print(f"\nModelo: {modelo_nome}")
        print(f"Erro Quadrático Médio (MSE) em %: {mse_percentual:.2f}%")
        print(f"R² em %: {r2_percentual:.2f}%")

    # Gráfico de MSE por modelo
    modelos = list(resultados.keys())
    mse_percentuais = [(resultados[m][0] / y.mean()) * 100 for m in modelos]

    plt.figure(figsize=(10, 6))
    sns.barplot(x=modelos, y=mse_percentuais, palette="magma", legend=False)
    plt.title("Erro Quadrático Médio (MSE %) por Modelo")
    plt.ylabel("MSE (%)")
    plt.xlabel("Modelos")
    plt.xticks(rotation=15)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(figures_dir, "mse_por_modelo.png"))

    print("\nGráfico de MSE salvo na pasta 'figures/'.")

    # Heatmap (em construção)
    df_mapa = df[['estado', 'codigo_ibge', 'casos']].copy()
    df_mapa = df_mapa.groupby(['estado', 'codigo_ibge']).sum().reset_index()
    print(df_mapa.head())
